In [1]:
import time
from queue import Queue
from threading import Thread, Event
from random import randint

# Shared queue to simulate a CAN bus
can_bus = Queue()
# Event to signal when the simulation should stop
running = Event()
running.set()

# Simulate a CAN message
class CANMessage:
    def __init__(self, arbitration_id, data):
        self.arbitration_id = arbitration_id
        self.data = data

    def __repr__(self):
        return f"ID: {hex(self.arbitration_id)}, Data: {self.data}"

# Sender Node (Legitimate Node)
def sender():
    while running.is_set():
        msg = CANMessage(arbitration_id=0x123, data=[randint(0, 255) for _ in range(4)])
        can_bus.put(msg)  # Send to the bus
        print(f"Sender: Sent {msg}")
        time.sleep(1)

# Receiver Node (Legitimate Node)
def receiver():
    while running.is_set():
        if not can_bus.empty():
            msg = can_bus.get()  # Receive from the bus
            print(f"Receiver: Received {msg}")
        time.sleep(0.5)

# Man-in-the-Middle (MitM) Attack
def mitm_attack():
    while running.is_set():
        if not can_bus.empty():
            # Intercept a legitimate message
            msg = can_bus.get()
            print(f"Attacker (MitM): Intercepted {msg}")
            
            # Modify the message
            malicious_msg = CANMessage(arbitration_id=msg.arbitration_id, data=[0xFF] * 4)
            can_bus.put(malicious_msg)  # Inject modified message
            print(f"Attacker (MitM): Injected {malicious_msg}")
        time.sleep(2)

# Replay Attack
def replay_attack(captured_messages):
    while running.is_set():
        if captured_messages:
            # Replay a random captured message
            msg = captured_messages[randint(0, len(captured_messages) - 1)]
            can_bus.put(msg)
            print(f"Attacker (Replay): Replayed {msg}")
        time.sleep(3)

# Denial of Service (DoS) Attack
def dos_attack():
    while running.is_set():
        for _ in range(5):  # Flood the bus with high-priority messages
            msg = CANMessage(arbitration_id=0x7FF, data=[0x00] * 8)
            can_bus.put(msg)
            print(f"Attacker (DoS): Flooded {msg}")
        time.sleep(5)

# Main Simulation
if __name__ == "__main__":
    # List to capture legitimate messages for Replay Attack
    captured_messages = []

    # Start legitimate sender and receiver threads
    sender_thread = Thread(target=sender)
    receiver_thread = Thread(target=receiver)

    # Start attack threads
    mitm_thread = Thread(target=mitm_attack)
    replay_thread = Thread(target=replay_attack, args=(captured_messages,))
    dos_thread = Thread(target=dos_attack)

    sender_thread.start()
    receiver_thread.start()
    mitm_thread.start()
    replay_thread.start()
    dos_thread.start()

    # Capture messages from sender
    try:
        time_limit = 30  # Run the simulation for 30 seconds
        start_time = time.time()
        while time.time() - start_time < time_limit:
            if not can_bus.empty():
                msg = can_bus.get()
                print(f"Captured for Replay: {msg}")
                captured_messages.append(msg)  # Save for Replay Attack
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("Simulation stopped manually.")
    finally:
        # Signal all threads to stop
        running.clear()
        sender_thread.join()
        receiver_thread.join()
        mitm_thread.join()
        replay_thread.join()
        dos_thread.join()
        print("All threads stopped.")


Sender: Sent ID: 0x123, Data: [113, 190, 31, 190]
Receiver: Received ID: 0x123, Data: [113, 190, 31, 190]
Attacker (DoS): Flooded ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Attacker (DoS): Flooded ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Attacker (DoS): Flooded ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Attacker (DoS): Flooded ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Attacker (DoS): Flooded ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Captured for Replay: ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Receiver: Received ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Captured for Replay: ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Sender: Sent ID: 0x123, Data: [244, 44, 61, 246]
Receiver: Received ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Captured for Replay: ID: 0x7ff, Data: [0, 0, 0, 0, 0, 0, 0, 0]
Receiver: Received ID: 0x123, Data: [244, 44, 61, 246]
Sender: Sent ID: 0x123, Data: [21, 233, 225, 252]
Attacker (MitM): Intercepted ID: 0x123, Data: [21, 233, 225, 252]
Attacker (MitM): Injected ID: 0x1